In [1]:
import time
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
import time
from IPython.display import clear_output
import requests
import pandas as pd
import pandas_ta as ta
try:
    import talib
    USE_TALIB = True
except ImportError:
    USE_TALIB = False
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from plotly.subplots import make_subplots



import plotly.graph_objects as go

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

# https://github.com/polygon-io/client-python/blob/master/examples/websocket/stocks-ws.py

In [2]:
def fetch_polygon_data(ticker: str, api_key: str,
                              start_date: date,
                              end_date: date,
                              start_time: dt_time,
                              end_time: dt_time = dt_time(16, 0),
                              multiplier: int = 1,
                              timespan: str = "minute",
                              limit: int = 50000) -> pd.DataFrame:
    """
    Fetches intraday minute-level OHLCV data for a given ticker from Polygon.io.

    Args:
        ticker (str): Stock ticker symbol (e.g., "TSLA").
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch data.
        end_date (date): The date for which to end fetch data.
        start_time (datetime.time): Start of time window in EST.
        end_time (datetime.time): End of time window in EST.
        multiplier (int): Interval multiplier (default 1 minute).
        timespan (str): Time unit ("minute", "hour", etc.).
        limit (int): Maximum number of results to retrieve.

    Returns:
        pd.DataFrame: A filtered DataFrame with renamed columns and timestamps in EST.
    """
    ET_ZONE = ZoneInfo("America/New_York")

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,
        "adjusted": "true",
        "sort": "asc",
        "limit": limit
    }

    session = requests.Session()
    response = session.get(url, params=params)
    response.raise_for_status()

    bars = response.json().get("results", [])
    df = pd.DataFrame(bars)

    if df.empty:
        return pd.DataFrame()  # Return empty DataFrame if no data

    df["TIME_UTC"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df["TIME_EST"] = df["TIME_UTC"].dt.tz_convert(ET_ZONE)
    del df["TIME_UTC"]

    # Define time window
    start_dt = datetime.combine(start_date, start_time, tzinfo=ET_ZONE)
    end_dt = datetime.combine(start_date, end_time, tzinfo=ET_ZONE)

    # Filter and rename
    df = df[(df["TIME_EST"] >= start_dt) & (df["TIME_EST"] <= end_dt)].copy()
    df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close",
        "v": "volume",
        "n": "trades",
        "vw": "vwap"
    }, inplace=True)

    return df


In [3]:

def add_candlestick_patterns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds all TA-Lib candlestick pattern columns and a summary column with detected patterns.
    
    Returns:
        DataFrame with candlestick pattern columns and a 'Detected_Patterns' summary column.
    """
    # Build dictionary of candlestick pattern functions
    cdl_patterns = {
        func: getattr(talib, func)
        for func in dir(talib)
        if func.startswith("CDL")
    }

    # Apply each pattern function to the OHLC data
    for name, pattern_func in cdl_patterns.items():
        try:
            df[name] = pattern_func(df['open'], df['high'], df['low'], df['close'])
        except Exception as e:
            print(f"⚠️ Could not compute {name}: {e}")
            df[name] = 0  # Fallback to 0 if something fails

    # Detect and summarize patterns per row
    def detect_patterns(row):
        patterns = []
        for name in cdl_patterns:
            val = row[name]
            if val != 0:
                direction = "Bullish" if val > 0 else "Bearish"
                strength = abs(val)
                patterns.append(f"{name} ({direction}, Strength={strength})")
        return ', '.join(patterns) if patterns else None

    # Add summary column
    df["Detected_Patterns"] = df.apply(detect_patterns, axis=1)

    return df

In [4]:

def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
    Assumes columns: 'open', 'high', 'low', 'close', 'volume'

    Returns:
        DataFrame with new columns added.
    """

    # ----------------- add candlestick patterns -----------------
    df = add_candlestick_patterns(df)
    # ------------------------------------------------------------

    # Bollinger Bands
    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BB_up'], df['BB_dn'] = up, dn

    # VWAP (Cumulative approximation)
    df["VWAP_Cum"] = (df["close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # EMA, MACD, ATR, SMA
    # df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    # df['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
    # df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    # df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    # df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)


        # EMA, MACD, ATR, SMA
    df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)
    df['EMA20'] = talib.EMA(df.close.values, timeperiod=20)
    df['EMA50'] = talib.EMA(df.close.values, timeperiod=50)
    df['EMA200'] = talib.EMA(df.close.values, timeperiod=200)



    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    df['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
    df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    df['SMA20'] = talib.SMA(df.close.values, timeperiod=20)
    df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

    # Candlestick pattern: Marubozu
    df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df.open.values, df.high.values, df.low.values, df.close.values)
    df['Bullish_marubozu'] = df['CDLMARUBOZU'] == 100
    df['Bearish_marubozu'] = df['CDLMARUBOZU'] == -100

    # RSI
    df['RSI'] = talib.RSI(df.close.values, timeperiod=14)

    # ----------------- Buy Conditions -----------------

    # B1: Price ≥ SMA5
    df['B_1'] = df['close'] >= df['SMA5']

    # B2: MACD histogram increasing & MACD strong
    df['MACD_diff'] = df['MACD'] - df['Signal']
    df['MACD_diff_avg'] = df['MACD_diff'].rolling(15).mean()
    df['MACD_strong'] = df['MACD_diff'] > 2 * df['MACD_diff_avg']
    df['B_2'] = (df['MACD_hist'].diff() > 0) & df['MACD_strong']

    # B3: RSI > smoothed RSI
    df['RSI_smooth'] = df['RSI'].ewm(span=3, adjust=False).mean()
    df['B_3'] = df['RSI'] > df['RSI_smooth']

    # B4: Volume spike
    df['Vol_avg20'] = df['volume'].rolling(20).mean()
    df['B_4'] = df['volume'] > 1.5 * df['Vol_avg20']

    # B5: Avoid bad momentum patterns
    df['Cond_MACD_hist_avoid'] = (df['MACD_hist'] > 0) & (df['MACD_hist'].diff() < 0)
    df['Cond_RSI_falling'] = df['RSI'].diff() < 0
    df['Cond_price_lower'] = df['close'] < df['close'].shift(1)
    df['B_5'] = df[['Cond_MACD_hist_avoid', 'Cond_RSI_falling', 'Cond_price_lower']].any(axis=1)

    # B6: Bullish marubozu breakout above recent high with follow-through
    N = 10
    df['RecentHigh'] = df['high'].shift(1).rolling(N).max()
    df['Breakout_follow_through'] = (
        (df['high'] > df['RecentHigh']) &
        (df['close'] > df['RecentHigh']) &
        (df['close'] > df['open'])
    )
    df['B_6'] = df['Bullish_marubozu'] & df['Breakout_follow_through']

    # Final Entry Signal
    df['Entry_Signal'] = df['B_1'] & df['B_2'] & df['B_3'] & ~df['B_5'] & df['B_6']

    return df

In [5]:
from datetime import date
import time
from typing import List, Dict
import pandas as pd

def fetch_multiple_tickers_data(ticker: List[str],
                                api_key: str,
                                start_date: date,
                                end_date: date,
                                start_time: dt_time,
                                end_time: dt_time,
                                multiplier: int,
                                timespan: str,
                                limit: int ,
                                delay: float = 0.25) -> Dict[str, pd.DataFrame]:
    """
    Fetch minute-level Polygon data for multiple tickers on a given date.

    Args:
        ticker (List[str]): List of stock ticker symbols.
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch intraday data.
        delay (float): Seconds to wait between API calls (rate limit buffer).

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of ticker symbol → DataFrame.
    """
    all_data = {}

    for ticker in ticker:
        try:
            df = fetch_polygon_data(
                ticker=ticker,
                api_key=API_KEY,
                start_date=start_date,
                end_date=end_date,
                start_time=start_time,
                end_time=end_time,
                multiplier=multiplier,
                timespan=timespan,
                limit=limit
                )


            if not df.empty:
                
                df = add_indicators_and_signals(df)

                all_data[ticker] = df

            print(f"✅ {ticker} - fetched {len(df)} rows")

        except Exception as e:
            print(f"❌ Error fetching {ticker}: {e}")

        time.sleep(delay) # ⏱️ Avoid hitting rate limits

    return all_data


In [6]:
# Add example 100 tickers

TICKERS = ["AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA", "META", "NFLX", "INTC", "AMD", 
           "BA", "CRM", "DIS", "ADBE", "PYPL", "CSCO", "PEP", "KO", "T", "VZ",
           "WMT", "COST", "MCD", "NKE", "QCOM", "ORCL", "IBM", "GE", "CAT", "HON",
           "JNJ", "PFE", "MRK", "LLY", "ABBV", "BMY", "CVX", "XOM", "SLB", "COP",
           "JPM", "BAC", "WFC", "GS", "MS", "BLK", "TGT", "LOW", "HD", "SBUX",
           "UBER", "LYFT", "ABNB", "SHOP", "SQ", "ROKU", "ZM", "PLTR", "SNOW", "NET",
           "PANW", "CRWD", "ZS", "OKTA", "DDOG", "DOCU", "F", "GM", "RIVN", "LCID",
           "TSM", "ASML", "AVGO", "TXN", "MU", "INTU", "ADSK", "EA", "ATVI", "TTWO",
           "NOK", "ERIC", "V", "MA", "AXP", "PYPL", "SOFI", "HOOD", "ROBIN", "COIN",
           "SPOT", "BIDU", "BABA", "JD", "PDD", "TME", "IQ", "YUMC", "NTES", "LI"]

# TICKERS = ["SRFM","PLTR","AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA"]

In [ ]:



# Date range 
start_date = date(2025, 7, 11)
end_date = date(2025, 7, 11)

# Time window for filtering intraday bars (Eastern Time)
start_time = dt_time(0, 0)   # 4:00 AM ET
end_time = dt_time(16, 0)    # 4:00 PM ET
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Granularity of data
multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
timespan = "minute"          #  "second", "minute", "hour", "day", etc.

# Max number of bars to retrieve per request (Polygon max = 50,000)
limit = 50000

# ⬇️ Fetch intraday data for multiple tickers from Polygon
all_data_minute = fetch_multiple_tickers_data(
    ticker=TICKERS,
    api_key=API_KEY,
    start_date=start_date,
    end_date=end_date,
    start_time=start_time,
    end_time=end_time,
    multiplier=multiplier,
    timespan=timespan,
    limit=limit
)

✅ AAPL - fetched 607 rows
✅ MSFT - fetched 459 rows
✅ GOOG - fetched 499 rows
✅ TSLA - fetched 715 rows
✅ AMZN - fetched 630 rows
✅ NVDA - fetched 715 rows
✅ META - fetched 451 rows
✅ NFLX - fetched 427 rows
✅ INTC - fetched 619 rows
✅ AMD - fetched 669 rows
✅ BA - fetched 443 rows
✅ CRM - fetched 424 rows
✅ DIS - fetched 415 rows
✅ ADBE - fetched 403 rows
✅ PYPL - fetched 435 rows
✅ CSCO - fetched 416 rows
✅ PEP - fetched 417 rows
✅ KO - fetched 439 rows
✅ T - fetched 433 rows
✅ VZ - fetched 442 rows
✅ WMT - fetched 474 rows
✅ COST - fetched 405 rows
✅ MCD - fetched 401 rows
✅ NKE - fetched 438 rows
✅ QCOM - fetched 430 rows
✅ ORCL - fetched 466 rows
✅ IBM - fetched 400 rows
✅ GE - fetched 397 rows
✅ CAT - fetched 394 rows
✅ HON - fetched 391 rows
✅ JNJ - fetched 404 rows
✅ PFE - fetched 507 rows
✅ MRK - fetched 428 rows
✅ LLY - fetched 396 rows
✅ ABBV - fetched 401 rows
✅ BMY - fetched 415 rows
✅ CVX - fetched 405 rows
✅ XOM - fetched 407 rows
✅ SLB - fetched 417 rows
✅ COP - fetched

In [ ]:

TICKERS_GOOD = ["NVDA", "GOOG","AMZN","PLTR","AVGO","SOFI","HOOD"]
TICKERS_BAD = ["ROKU","TME","LCID","IQ" ]

df=all_data_minute[TICKER]

# ── PLOTLY SUBPLOTS ───────────────────────────────────────────────────────────
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    row_heights=[0.5,0.2,0.2,0.1],
    vertical_spacing=0,
    subplot_titles=("Price + MA5/EMA9/VWAP/BB","RSI(5)","MACD","Volume")
)

# ─ BB lines + fill (light, and placed first) ────────────────────────────────
fig.add_trace(go.Scatter(
    x=df.index, y=df["BB_up"],
    line=dict(color="gray", dash="dash"),
    name="BB Upper"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df["BB_dn"],
    fill="tonexty",
    fillcolor="rgba(189,183,107,0.15)",  # much lighter khaki
    line=dict(color="gray", dash="dash"),
    name="BB Lower"
), row=1, col=1)

# ─ Price Candles ─────────────────────────────────────────────────────────────
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"], high=df["high"],
    low=df["low"], close=df["close"],
    increasing_line_color="YellowGreen",
    decreasing_line_color="darkred",
    name="Price"
), row=1, col=1)

# ─ MA/EMA/VWAP ────────────────────────────────────────────────────────────────
fig.add_trace(go.Scatter(x=df.index, y=df["EMA9"],  line=dict(color="red"),     name="EMA9"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA50"], line=dict(color="gold"),        name="EMA50"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA50"], line=dict(color="magenta"),        name="SMA50"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA5"],  line=dict(color="fuchsia"),     name="SMA5"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA200"], line=dict(color="blue"),        name="EMA200"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA200"], line=dict(color="black"),        name="SMA200"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["VWAP_Cum"], line=dict(color="Maroon"),  name="VWAP"),row=1,col=1)

# ─ RSI + Oversold/Overbought Bands ─────────────────────────────────────────────
fig.add_trace(go.Scatter(
    x=df.index, y=df["RSI"], line=dict(color="rebeccapurple"), name="RSI(5)"
), row=2, col=1)
fig.add_hrect(y0=70, y1=100, fillcolor="darkkhaki", opacity=0.3, row=2, col=1)
fig.add_hrect(y0=0,  y1=30,  fillcolor="darkkhaki", opacity=0.3, row=2, col=1)

# ─ MACD & Histogram ──────────────────────────────────────────────────────────
hist_cols = df["MACD_hist"].apply(lambda x: "green" if x>=0 else "red")
fig.add_trace(go.Bar(x=df.index, y=df["MACD_hist"], marker_color=hist_cols, name="MACD Hist"), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["MACD"],   line=dict(color="pink"),  name="MACD"), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["Signal"], line=dict(color="black"), name="Signal"),row=3,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["ATR"], line=dict(color="red"), name="ATR"),row=3,col=1)

# ─ Volume Overlay ─────────────────────────────────────────────────────────────
# fig.add_trace(go.Bar(x=df.index, y=df["vol_up"], name="Vol > avg", marker_color="olive",  offsetgroup="vol"), row=4, col=1)
# fig.add_trace(go.Bar(x=df.index, y=df["vol_dn"], name="Vol ≤ avg", marker_color="red",   offsetgroup="vol"), row=4, col=1)

# ─ ADDED BY FARNAZ ─────────────────────────────────────────────────────────────
# for ts in entry_times:
# for ts in clean_entry_times:
#     fig.add_vline(x=ts, line=dict(color="green", width=2, dash="dash"),layer="above")    
    
    
# # for ts in exit_times:
# for ts in clean_exit_times:
#     fig.add_vline(x=ts, line=dict(color="red", width=2, dash="dash"),layer="above")
    
# ── FINAL LAYOUT ──────────────────────────────────────────────────────────────
fig.update_layout(
    title=f"{TICKER} Intraday {start_date}",
    height=700, width=1000,
    barmode="overlay",
    xaxis_rangeslider_visible=False,
    margin=dict(l=50,r=20,t=80,b=40),
    paper_bgcolor="white", plot_bgcolor="white",
    legend=dict(orientation="h", y=-0.15, x=0.5, xanchor="center")
)

# grids & axis titles
fig.update_xaxes(showgrid=True, gridcolor="lightgray")
fig.update_yaxes(showgrid=True, gridcolor="lightgray")
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="RSI",   row=2, col=1, range=[0,100])
fig.update_yaxes(title_text="MACD",  row=3, col=1)
fig.update_yaxes(title_text="Volume",row=4, col=1)


fig.show()

In [33]:
import numpy as np
import talib

TICKERS_GOOD = ["NVDA", "GOOG", "AMZN", "PLTR", "AVGO", "SOFI", "HOOD"]
TICKERS_BAD  = ["ROKU", "TME", "LCID", "IQ"]
ALL_TICKERS  = TICKERS_GOOD + TICKERS_BAD

for TICKER in ALL_TICKERS:
    print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📊 Ticker: {TICKER}")

    df = all_data_minute[TICKER].copy()

    # ─── Indicators ───
    df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
    df['SMA50'] = talib.SMA(df['close'], timeperiod=50)
    df['SMA200'] = talib.SMA(df['close'], timeperiod=200)
    upper, mid, lower = talib.BBANDS(df['close'], timeperiod=20)
    df['bb_width'] = upper - lower

    df['trend_slope'] = df['close'].rolling(50).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True)
    flat_trend_periods = (df['trend_slope'].abs() < 0.0005).sum()

    df['cross'] = np.where(df['close'] > df['EMA50'], 1, 0)
    df['cross_change'] = df['cross'].diff().fillna(0)
    cross_count = df['cross_change'].abs().sum()

    narrow_bb_periods = (df['bb_width'] < 0.005 * df['close']).sum()

    df['upper_wick'] = df['high'] - df[['close', 'open']].max(axis=1)
    df['lower_wick'] = df[['close', 'open']].min(axis=1) - df['low']
    spike_count = ((df['upper_wick'] > 0.02 * df['close']) | (df['lower_wick'] > 0.02 * df['close'])).sum()

    golden_cross = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) <= df['SMA200'].shift(1))
    flat_after_cross = df['trend_slope'].rolling(10).mean().abs() < 0.001
    bad_golden_cross = (golden_cross & flat_after_cross).sum()

    discard = (
        cross_count > 30 and
        flat_trend_periods > 100 and
        narrow_bb_periods > 100 and
        spike_count > 20 and
        bad_golden_cross >= 2
    )

    # ─── Print Formatted Results ───
    print(f"🔁 EMA50 Crossings: {cross_count}")
    print(f"📉 Flat Trend Periods: {flat_trend_periods}")
    print(f"📏 Narrow BB Widths (<0.5%): {narrow_bb_periods}")
    print(f"⚡ Wick Spikes >2%: {spike_count}")
    print(f"💔 Failed Golden Crosses: {bad_golden_cross}")
    if discard:
        print(f"🚫 Verdict: Choppy/sideways stock — DISCARD.")
    else:
        print(f"✅ Verdict: Trend is OK — KEEP.")



━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Ticker: NVDA
🔁 EMA50 Crossings: 68.0
📉 Flat Trend Periods: 43
📏 Narrow BB Widths (<0.5%): 594
⚡ Wick Spikes >2%: 0
💔 Failed Golden Crosses: 0
✅ Verdict: Trend is OK — KEEP.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Ticker: GOOG
🔁 EMA50 Crossings: 34.0
📉 Flat Trend Periods: 20
📏 Narrow BB Widths (<0.5%): 393
⚡ Wick Spikes >2%: 0
💔 Failed Golden Crosses: 0
✅ Verdict: Trend is OK — KEEP.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Ticker: AMZN
🔁 EMA50 Crossings: 44.0
📉 Flat Trend Periods: 12
📏 Narrow BB Widths (<0.5%): 534
⚡ Wick Spikes >2%: 0
💔 Failed Golden Crosses: 0
✅ Verdict: Trend is OK — KEEP.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Ticker: PLTR
🔁 EMA50 Crossings: 68.0
📉 Flat Trend Periods: 21
📏 Narrow BB Widths (<0.5%): 452
⚡ Wick Spikes >2%: 0
💔 Failed Golden Crosses: 0
✅ Verdict: Trend is OK — KEEP.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Ticker: AVGO
🔁 EMA50 Crossings: 62.0
📉 Flat Trend Periods: 4
📏 Narrow BB Widths (<0.5%): 318
⚡ Wic

In [34]:
import pandas as pd
import numpy as np
import talib
from IPython.display import display, HTML

TICKERS_GOOD = ["NVDA", "GOOG", "AMZN", "PLTR", "AVGO", "SOFI", "HOOD"]
TICKERS_BAD  = ["ROKU", "TME", "LCID", "IQ"]
ALL_TICKERS  = TICKERS_GOOD + TICKERS_BAD

results = []

for TICKER in ALL_TICKERS:
    df = all_data_minute[TICKER].copy()

    df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
    df['SMA50'] = talib.SMA(df['close'], timeperiod=50)
    df['SMA200'] = talib.SMA(df['close'], timeperiod=200)
    upper, mid, lower = talib.BBANDS(df['close'], timeperiod=20)
    df['bb_width'] = upper - lower

    df['trend_slope'] = df['close'].rolling(50).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True)
    flat_trend_periods = (df['trend_slope'].abs() < 0.0005).sum()

    df['cross'] = np.where(df['close'] > df['EMA50'], 1, 0)
    df['cross_change'] = df['cross'].diff().fillna(0)
    cross_count = df['cross_change'].abs().sum()

    narrow_bb_periods = (df['bb_width'] < 0.005 * df['close']).sum()

    df['upper_wick'] = df['high'] - df[['close', 'open']].max(axis=1)
    df['lower_wick'] = df[['close', 'open']].min(axis=1) - df['low']
    spike_count = ((df['upper_wick'] > 0.02 * df['close']) | (df['lower_wick'] > 0.02 * df['close'])).sum()

    golden_cross = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) <= df['SMA200'].shift(1))
    flat_after_cross = df['trend_slope'].rolling(10).mean().abs() < 0.001
    bad_golden_cross = (golden_cross & flat_after_cross).sum()

    verdict = "DISCARD" if (
        cross_count > 30 and
        flat_trend_periods > 100 and
        narrow_bb_periods > 100 and
        spike_count > 20 and
        bad_golden_cross >= 2
    ) else "KEEP?"

    results.append({
        "Ticker": TICKER,
        "Cross_Count": cross_count,
        "Flat_Trend": flat_trend_periods,
        "Narrow_BB": narrow_bb_periods,
        "Spikes": spike_count,
        "Failed_Golden_Cross": bad_golden_cross,
        "Verdict": verdict
    })

# Convert to DataFrame
df_result = pd.DataFrame(results)

# Display full scrollable table (prevent truncation)
display(HTML(df_result.to_html(index=False, max_rows=1000, max_cols=100, notebook=True)))


Ticker,Cross_Count,Flat_Trend,Narrow_BB,Spikes,Failed_Golden_Cross,Verdict
NVDA,68.0,43,594,0,0,KEEP?
GOOG,34.0,20,393,0,0,KEEP?
AMZN,44.0,12,534,0,0,KEEP?
PLTR,68.0,21,452,0,0,KEEP?
AVGO,62.0,4,318,0,0,KEEP?
SOFI,54.0,61,199,0,1,KEEP?
HOOD,56.0,25,258,0,0,KEEP?
ROKU,32.0,10,184,1,0,KEEP?
TME,25.0,78,243,0,0,KEEP?
LCID,135.0,463,1,0,2,KEEP?


In [ ]:
| Indicator                              | Category                   | Description                                                                     |
| -------------------------------------- | -------------------------- | ------------------------------------------------------------------------------- |
| **ADX** (Average Directional Index)    | Trend Strength             | Measures the strength of a trend (not direction)                                |
| **+DI / -DI** (Directional Indicators) | Trend Direction            | Part of ADX; show bullish or bearish directional strength                       |
| **Aroon / Aroon Oscillator**           | Trend Strength & Timing    | Measures how recently highs or lows occurred (TA-Lib: `AROON`)                  |
| **MACD Histogram Height**              | Trend Strength & Momentum  | Taller bars = stronger trend; flattening bars = weakening trend                 |
| **Slope of EMA / SMA**                 | Custom / Trend Angle       | A sharply rising/falling EMA (e.g. EMA50 or EMA200) implies a strong trend      |
| **Rate of Change (ROC)**               | Momentum / Trend           | Measures speed of price movement, useful to infer trend strength                |
| **Moving Average Spread**              | Custom                     | Distance between fast and slow MAs (e.g., EMA20 vs EMA50) can indicate strength |
| **Z-Score of Price**                   | Custom Volatility Strength | Shows how far price is from average in terms of standard deviation              |
| **RSI Trend** (RSI > 50 or < 50)       | Indirect                   | Persistent RSI above 50 often aligns with bullish trend strength                |
